# Shohei Ohtani Pitching Stats

- 大谷翔平の投手成績を可視化する例
- サンプルデータはkm/h & m 単位になっていますが, [オリジナルデータ](https://baseballsavant.mlb.com/statcast_search)はmile/h & feetなのでご注意ください

In [24]:
import pandas as pd

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [25]:
# データ読み込み. game_dateは予めデータ型を変更

df = pd.read_csv('./dataset/20220919_baseball-savant-shohei-ohtani-pitch.csv', header=0)

df['game_date'] = pd.to_datetime(df['game_date'])
df.head(10)

,game_date,game_month,game_pk,inning,pitcher_name,p_throws,batter_name,stand,at_bat_number,pitch_number,pitch_name,effective_speed,release_spin_rate,release_extension,type,events,description,des,zone,bb_type,launch_speed_angle,hit_location,launch_speed,launch_angle,hit_distance_sc,pfx_x,pfx_z,plate_x,plate_z,release_pos_x,release_pos_z,effective_speed_kh,release_extension_cm,pfx_x_cm,pfx_z_cm,plate_x_cm,plate_z_cm,release_pos_x_cm,release_pos_z_cm,launch_speed_angle_category,launch_speed_kh,hit_distance_sc_m,hc_x,hc_y,hc_x_0,hc_y_0,hc_x_ft,hc_y_ft,hc_x_0_ft,hc_y_0_ft,hc_x_m,hc_y_m,hc_x_0_m,hc_y_0_m,events_summary
0,2022-09-17,9,663426,1,"Ohtani, Shohei",R,"Crawford, J.P.",L,1,1,Cutter,91.7,2350.0,6.8,S,NaN,called_strike,"J.P. Crawford grounds out, shortstop Luis Reng...",8.0,NaN,NaN,NaN,NaN,NaN,NaN,0.25,0.65,-0.14,1.61,-2.43,5.86,146.72,207.26,7.62,19.81,-4.27,49.07,-74.07,178.61,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
1,2022-09-17,9,663426,1,"Ohtani, Shohei",R,"Crawford, J.P.",L,1,2,Cutter,94.0,2329.0,6.7,B,NaN,ball,"J.P. Crawford grounds out, shortstop Luis Reng...",11.0,NaN,NaN,NaN,NaN,NaN,NaN,0.11,0.76,-0.72,3.45,-2.34,5.87,150.40,204.22,3.35,23.16,-21.95,105.16,-71.32,178.92,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
2,2022-09-17,9,663426,1,"Ohtani, Shohei",R,"Crawford, J.P.",L,1,3,Slider,87.5,2473.0,6.7,S,NaN,foul,"J.P. Crawford grounds out, shortstop Luis Reng...",11.0,NaN,NaN,NaN,NaN,NaN,NaN,0.74,0.91,-0.45,3.42,-2.38,5.84,140.00,204.22,22.56,27.74,-13.72,104.24,-72.54,178.00,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
3,2022-09-17,9,663426,1,"Ohtani, Shohei",R,"Crawford, J.P.",L,1,4,Slider,87.5,2485.0,6.7,S,NaN,foul,"J.P. Crawford grounds out, shortstop Luis Reng...",13.0,NaN,NaN,NaN,78.4,-37.0,2.0,0.51,-0.40,-0.53,1.32,-1.84,6.29,140.00,204.22,15.54,-12.19,-16.15,40.23,-56.08,191.72,Unknown,125.44,0.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
4,2022-09-17,9,663426,1,"Ohtani, Shohei",R,"Crawford, J.P.",L,1,5,4-Seam Fastball,99.3,2365.0,6.8,S,NaN,foul,"J.P. Crawford grounds out, shortstop Luis Reng...",1.0,NaN,NaN,NaN,99.4,-11.0,15.0,-0.23,1.22,-0.55,3.18,-2.05,6.01,158.88,207.26,-7.01,37.19,-16.76,96.93,-62.48,183.18,Unknown,159.04,4.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
5,2022-09-17,9,663426,1,"Ohtani, Shohei",R,"Crawford, J.P.",L,1,6,Split-Finger,91.1,1309.0,6.9,B,NaN,ball,"J.P. Crawford grounds out, shortstop Luis Reng...",14.0,NaN,NaN,NaN,NaN,NaN,NaN,-0.92,0.03,0.20,0.47,-1.83,6.13,145.76,210.31,-28.04,0.91,6.10,14.33,-55.78,186.84,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
6,2022-09-17,9,663426,1,"Ohtani, Shohei",R,"Crawford, J.P.",L,1,7,Cutter,91.1,2532.0,6.8,B,NaN,ball,"J.P. Crawford grounds out, shortstop Luis Reng...",13.0,NaN,NaN,NaN,NaN,NaN,NaN,0.04,0.26,-1.09,1.63,-2.10,6.13,145.76,207.26,1.22,7.92,-33.22,49.68,-64.01,186.84,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
7,2022-09-17,9,663426,1,"Ohtani, Shohei",R,"Crawford, J.P.",L,1,8,4-Seam Fastball,98.6,2238.0,6.8,X,field_out,hit_into_play,"J.P. Crawford grounds out, shortstop Luis Reng...",4.0,ground_ball,2.0,NaN,90.5,-15.0,9.0,-0.30,1.13,-0.62,2.48,-2.21,5.86,157.76,207.26,-9.14,34.44,-18.90,75.59,-67.36,178.61,Topped,144.80,2.74,104.56,149.6,-20.86,48.67,261.4,374.0,-52.15,121.68,79.67,114.0,-15.9,37.09,other
8,2022-09-17,9,663426,1,"Ohtani, Shohei",R,"France, Ty",R,2,1,Sinker,97.5,1817.0,7.0,B,NaN,ball,Ty France strikes out swinging.,13.0,NaN,NaN,NaN,NaN,NaN,NaN,-1.39,0.57,-0.28,0.74,-2.19,5.82,156.00,213.36,-42.37,17.37,-8.53,22.56,-66.75,177.39,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
9,2022-09-17,9,663426,1,"Ohtani, Shohei",R,"France, Ty",R,2,2,Slider,87.2,2401.0,6.8,S,NaN,called_strike,Ty France strikes out swinging.,6.0,NaN,NaN,NaN,NaN,NaN,NaN,1.07,0.56,0.53,2.36,-2.30,5.80,139.52,207.26,32.61,17.07,16.15,71.93,-70.10,176.78,Unknown,NaN,NaN,NaN,NaN,NaN,N

In [26]:
# 念の為データ型を確認
df.dtypes

game_date                      datetime64[ns]
game_month                              int64
game_pk                                 int64
inning                                  int64
pitcher_name                           object
p_throws                               object
batter_name                            object
stand                                  object
at_bat_number                           int64
pitch_number                            int64
pitch_name                             object
effective_speed                       float64
release_spin_rate                     float64
release_extension                     float64
type                                   object
events                                 object
description                            object
des                                    object
zone                                  float64
bb_type                                object
launch_speed_angle                    float64
hit_location                      

## 投球データの可視化

- どこに投げられているか
- 球種毎, 結果毎にプロット

In [27]:
# グラフ描画用の関数

import plotly.express as px
import plotly.graph_objects as go

# ゾーンの横幅と縦幅
PLATE_ZONE_X = (-100, 100)
PLATE_ZONE_Y = (0, 225)

# ストライクゾーン定義（本当は上下可変ですがわかりやすいように固定にしています）
PLATE_STRIKE_ZONE_X = [-28.9, -28.9, 28.9, 28.9, -28.9]
PLATE_STRIKE_ZONE_Y = [48, 104, 104, 48, 48]

HOVER = [
    "game_date",
    "batter_name",
    "pitcher_name",
    "effective_speed_kh",
    "pitch_name",
    "hit_distance_sc_m",
    "launch_speed_kh",
    "launch_angle",
    "des",
    "description"
]
LEGEND = dict(
    yanchor="top",
    y=1,
    xanchor="left",
    x=0,
    orientation="h"
)
def strike_zone(name: str = 'strike zone') -> go.Scatter:
    return go.Scatter(
        name=name,
        x=PLATE_STRIKE_ZONE_X,
        y=PLATE_STRIKE_ZONE_Y,
        line_color='red'
    )


def plate(df: pd.DataFrame, title: str = 'Pitch on plate(Umpire view)',
            hover: str = 'pitch_name') -> go.Figure:
    fig = px.scatter(
        df.sort_values(hover),
        x="plate_x_cm",
        y="plate_z_cm",
        color=hover,
        hover_name=hover,
        hover_data=HOVER
    )
    fig.add_trace(strike_zone())
    fig.update_layout(
        title=title,
        autosize=False,
        width=800,
        height=800,
        legend=LEGEND,
        legend_title=None
    )
    fig.update_xaxes(range=PLATE_ZONE_X, title='plate_x(cm)', dtick=25)
    fig.update_yaxes(range=PLATE_ZONE_Y, title='plate_x(cm)', dtick=25)
    return fig


In [28]:
# 早速可視化してみる

_fig = plate(df)
_fig.show()

In [29]:
# 結果ごとに可視化

_fig = plate(df, hover='events')
_fig.show()

In [30]:
# 一試合に絞る. 9/17, 7回無失点の試合

_fig = plate(df[df['game_date'] == '2022-09-17'])
_fig.show()

## リリースした位置

- どこからリリースしたか, 高さと横位置を可視化
- キャッチャー（審判）から見た視点での可視化
  - TV中継みたいに投手の背中越しに見たい場合はX軸を逆転させると可能

In [31]:
def release(df: pd.DataFrame, title: str = 'Release Point View(Umpire view)',
            hover: str = 'pitch_name') -> go.Figure:
    fig = px.scatter(
        df.sort_values(hover),
        x="release_pos_x_cm",
        y="release_pos_z_cm",
        color=hover,
        hover_name=hover,
        hover_data=HOVER
    )
    fig.add_trace(strike_zone())
    fig.update_layout(
        title=title,
        autosize=False,
        width=800,
        height=800,
        legend=LEGEND,
        legend_title=None
    )
    fig.update_xaxes(range=(-125, 125), title='release_pos_x(cm)', dtick=25)
    fig.update_yaxes(range=(0, 275), title='release_pos_z(cm)', dtick=25)
    return fig


In [32]:
_fig = release(df[df['game_date'] == '2022-09-17'])
_fig.show()

# 投げたボールの比率

変化球の種類を円グラフで表す

In [33]:
def pie(df: pd.DataFrame, key: str, title: str) -> go.Figure:
    df_copy: pd.DataFrame = df.copy()
    # Key毎にサマって数える
    df_pie = df_copy[[key, 'game_date']].groupby(key).count().reset_index().sort_values(key)
    fig = px.pie(
        df_pie,
        values='game_date',
        names=key,
        color=key,
    )
    fig.update_layout(
        title=title,
        autosize=True,
        legend_title=None
    )
    return fig


In [34]:
_fig = pie(df, key='pitch_name', title='球種割合')
_fig.show()